# Data Preprocessing

In [2]:
# reading the data
en = []
with open("../data/raw/europarl-v7.fr-en.en", "r") as file:
    for line in file:
        en.append(line.strip())

fr = []
with open("../data/raw/europarl-v7.fr-en.fr", "r") as file:
    for line in file:
        fr.append(line.strip())

In [3]:
print(len(en))
print(len(fr))

2007723
2007723


In [4]:
print(en[:10])
print(fr[:10])

['Resumption of the session', 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.', "Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.", 'You have requested a debate on this subject in the course of the next few days, during this part-session.', "In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the European Union.", "Please rise, then, for this minute' s silence.", "(The House rose and observed a minute' s silence)", 'Madam President, on a point of order.', 'You will be aware from the press and television that there have been a num

In [5]:
# cleaning the data
import re

clean_en = []
clean_fr = []

for en_line, fr_line in zip(en, fr):
    en_line_clean = en_line.strip()
    fr_line_clean = fr_line.strip()

    if len(en_line_clean) == 0 or len(fr_line_clean) == 0:
        continue

    en_line_clean = re.sub(r"\s+", " ", en_line_clean)
    fr_line_clean = re.sub(r"\s+", " ", fr_line_clean)

    en_line_clean = en_line_clean.lower()
    fr_line_clean = fr_line_clean.lower()

    clean_en.append(en_line_clean)
    clean_fr.append(fr_line_clean)

In [6]:
print(len(clean_en))
print(len(clean_fr))

2002756
2002756


In [7]:
print(clean_en[:5])
print(clean_fr[:5])

['resumption of the session', 'i declare resumed the session of the european parliament adjourned on friday 17 december 1999, and i would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.', "although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.", 'you have requested a debate on this subject in the course of the next few days, during this part-session.', "in the meantime, i should like to observe a minute' s silence, as a number of members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the european union."]
['reprise de la session', 'je déclare reprise la session du parlement européen qui avait été interrompue le vendredi 17 décembre dernier et je vous renouvelle tous mes vux en espérant que vous avez passé de bonnes vacan

In [8]:
# length filtering to prevent vanishing/exploding gradients
# 2 conditions: if length > 100 or length ratio too large

filtered_en = []
filtered_fr = []

for en_line, fr_line in zip(clean_en, clean_fr):
    en_len = len(en_line.split())
    fr_len = len(fr_line.split())

    ratio = max(en_len, fr_len) / min(en_len, fr_len)

    if en_len > 100 or fr_len > 100 or ratio > 3:
        continue

    filtered_en.append(en_line)
    filtered_fr.append(fr_line)

In [9]:
print(len(filtered_en), len(filtered_fr))
print(filtered_en[:5])
print(filtered_fr[:5])

1996288 1996288
['resumption of the session', 'i declare resumed the session of the european parliament adjourned on friday 17 december 1999, and i would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.', "although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.", 'you have requested a debate on this subject in the course of the next few days, during this part-session.', "in the meantime, i should like to observe a minute' s silence, as a number of members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the european union."]
['reprise de la session', 'je déclare reprise la session du parlement européen qui avait été interrompue le vendredi 17 décembre dernier et je vous renouvelle tous mes vux en espérant que vous avez passé

In [10]:
# writing a combined corpus for the sentence piece model

with open("../data/spm/corpus.txt", "w") as file:
    for en, fr in zip(filtered_en, filtered_fr):
        file.write(en + "\n")
        file.write(fr + "\n")

In [11]:
spm_corpus = []

with open("../data/spm/corpus.txt", "r") as file:
    for line in file:
        spm_corpus.append(line)

print(len(spm_corpus))
print(spm_corpus[:5])

3992576
['resumption of the session\n', 'reprise de la session\n', 'i declare resumed the session of the european parliament adjourned on friday 17 december 1999, and i would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.\n', 'je déclare reprise la session du parlement européen qui avait été interrompue le vendredi 17 décembre dernier et je vous renouvelle tous mes vux en espérant que vous avez passé de bonnes vacances.\n', "although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.\n"]


In [ ]:
# training the spm tokenizer

import sentencepiece as spm

spm.SentencePieceTrainer.Train(
    input='../data/spm/corpus.txt',
    model_prefix='../data/spm/spm',
    vocab_size=16000,
    model_type='bpe',
    character_coverage=1.0,
    input_sentence_size=2000000,
    shuffle_input_sentence=True
)

In [13]:
import sentencepiece as spm

sp = spm.SentencePieceProcessor()
sp.load('../data/spm/spm.model')
print("Vocab size: ", sp.get_piece_size())

for i in range(30):
    print(i, sp.id_to_piece(i))

Vocab size:  16000
0 <unk>
1 <s>
2 </s>
3 ▁t
4 on
5 ▁d
6 es
7 en
8 ▁a
9 ▁p
10 ▁l
11 ▁c
12 in
13 ▁th
14 re
15 ti
16 ▁s
17 er
18 is
19 ou
20 an
21 ▁the
22 ent
23 ▁m
24 ▁de
25 it
26 ▁e
27 or
28 qu
29 tion


In [14]:
tokenized_en = []
tokenized_fr = []

for en, fr in zip(clean_en, clean_fr):
    tok_en = sp.encode(en, out_type=str)
    tok_fr = sp.encode(fr, out_type=str)

    tokenized_en.append(tok_en)
    tokenized_fr.append(tok_fr)

In [15]:
print(tokenized_en[:5])
print(tokenized_fr[:5])

[['▁res', 'umption', '▁of', '▁the', '▁session'], ['▁i', '▁declare', '▁resumed', '▁the', '▁session', '▁of', '▁the', '▁european', '▁parliament', '▁adjour', 'ned', '▁on', '▁friday', '▁17', '▁december', '▁1999,', '▁and', '▁i', '▁would', '▁like', '▁once', '▁again', '▁to', '▁wish', '▁you', '▁a', '▁happy', '▁new', '▁year', '▁in', '▁the', '▁hope', '▁that', '▁you', '▁enjoyed', '▁a', '▁pleas', 'ant', '▁f', 'es', 'tive', '▁period', '.'], ['▁although', ',', '▁as', '▁you', '▁will', '▁have', '▁seen', ',', '▁the', '▁d', 'read', 'ed', "▁'", 'm', 'ill', 'enn', 'ium', '▁b', 'ug', "'", '▁failed', '▁to', '▁material', 'ise', ',', '▁still', '▁the', '▁people', '▁in', '▁a', '▁number', '▁of', '▁countries', '▁suffered', '▁a', '▁series', '▁of', '▁natural', '▁disasters', '▁that', '▁truly', '▁were', '▁d', 'read', 'ful', '.'], ['▁you', '▁have', '▁requested', '▁a', '▁debate', '▁on', '▁this', '▁subject', '▁in', '▁the', '▁course', '▁of', '▁the', '▁next', '▁few', '▁days', ',', '▁during', '▁this', '▁part', '-', 'session

In [16]:
tokenized_id_en = [sp.piece_to_id(piece) for piece in tokenized_en]
tokenized_id_fr = [sp.piece_to_id(piece) for piece in tokenized_fr]

In [17]:
print(tokenized_id_en[:5])
print(tokenized_id_fr[:5])

[[218, 5411, 48, 21, 3876], [74, 10429, 11208, 21, 3876, 48, 21, 231, 443, 13802, 10182, 94, 11982, 3791, 4400, 7700, 62, 74, 359, 540, 2203, 731, 50, 1681, 446, 8, 5741, 643, 695, 39, 21, 1352, 91, 446, 13924, 8, 8662, 93, 32, 6, 214, 2597, 15804], [2490, 15802, 136, 446, 251, 198, 3460, 15802, 21, 5, 1125, 82, 1187, 15795, 169, 4032, 2373, 38, 1100, 15808, 6197, 50, 5021, 317, 15802, 1328, 21, 663, 39, 8, 1527, 48, 617, 8741, 8, 6700, 48, 4415, 7214, 91, 6331, 960, 5, 1125, 1061, 15804], [446, 198, 7608, 8, 1070, 94, 139, 1877, 39, 21, 1374, 48, 21, 1794, 1945, 4518, 15802, 2072, 139, 466, 15813, 9336, 15804], [39, 21, 11859, 15802, 74, 377, 540, 50, 8700, 8, 6607, 15808, 16, 7703, 15802, 136, 8, 1527, 48, 1196, 198, 7608, 15802, 94, 1888, 48, 282, 21, 4204, 2027, 15802, 1630, 820, 48, 21, 7381, 10638, 578, 15802, 39, 21, 2483, 617, 48, 21, 231, 410, 15804]]
[[6812, 24, 44, 3876], [171, 8637, 6812, 44, 3876, 132, 411, 217, 162, 2606, 477, 15328, 56, 11664, 3791, 4417, 2473, 66, 171, 

In [18]:
enc_in = tokenized_id_en[:]

dec_in = []
dec_tgt = []

for sentence in tokenized_id_fr:
    dec_in.append([1] + sentence)
    dec_tgt.append(sentence + [2])

In [19]:
print(enc_in[0])
print(dec_in[0])
print(dec_tgt[0])

[218, 5411, 48, 21, 3876]
[1, 6812, 24, 44, 3876]
[6812, 24, 44, 3876, 2]
